<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

## Загрузка данных

In [3]:
data = pd.read_csv("/datasets/insurance.csv")

In [4]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [6]:
numeric = ['Возраст', 'Зарплата', 'Члены семьи']
data[numeric].corr()

,Возраст,Зарплата,Члены семьи
Возраст,1.000000,-0.019093,-0.006692
Зарплата,-0.019093,1.000000,-0.030296
Члены семьи,-0.006692,-0.030296,1.000000


Корреляция между данными не такая сильна, она не вызывет мультиколлинеарности, поэтому можно использовать точную аналитическую форму линейной регрессии

In [5]:
X = data.drop("Страховые выплаты", axis=1)
y = data["Страховые выплаты"]

X.shape, y.shape

((5000, 4), (5000,))

In [6]:
INV = np.random.rand(4,4)

np.linalg.inv(INV)

array([[ 0.25420278,  0.54367821,  0.66309128, -1.09817012],
       [-1.87446782,  1.92891409,  0.63987127,  0.34615606],
       [ 1.2586406 , -0.31993337, -0.98504524,  0.41556198],
       [ 1.04039461, -2.05960641,  0.19240752,  0.5799617 ]])

In [7]:
def predict_and_r2_score(x, y):
    lr = LinearRegression()
    lr.fit(x, y)
    r2 = lr.score(x, y)
    return r2

In [8]:
# r2 для обычной матрицы признаков
predict_and_r2_score(X, y)

0.4249455028666801

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество модели не изменится

**Обоснование:** 
$$
a = Xw = X(X^T X)^{-1} X^T y
$$
$$
w' = ((XP)^TXP)^{-1}(XP)^Ty
$$
$$
a' = XPw' = XP((XP)^TXP)^{-1}(XP)^Ty =
$$
$$
= XP(P^TX^TXP)^{-1}P^TX^Ty =
$$
$$
= XPP^{-1}(P^TX^TX)^{-1}P^TX^Ty =
$$
$$
= X(P^TX^TX)^{-1}P^TX^Ty = 
$$
$$
= X(X^TX)^{-1}(P^T)^{-1}P^TX^Ty =
$$
$$
= X(X^TX)^{-1}X^Ty = a
$$

## Алгоритм преобразования

**Алгоритм**: <br>
1) Умножим матрицу признаков на 100
<br>
2) Умножим на обратимую матрицу

**Обоснование**:<br>
В данном случае $$X=100P$$

## Проверка алгоритма

In [9]:
X_new = (X * 100) @ INV

X_new.head()

,0,1,2,3
0,4.166317e+06,3.970897e+06,1.251595e+06,4.622305e+06
1,3.192566e+06,3.043026e+06,9.598433e+05,3.541346e+06
2,1.764481e+06,1.681826e+06,5.306492e+05,1.957041e+06
3,3.502031e+06,3.337737e+06,1.051316e+06,3.886051e+06
4,2.192696e+06,2.089864e+06,6.590552e+05,2.432326e+06


In [10]:
predict_and_r2_score(X_new, y)

0.4249455028666844

## Вывод

Качество лин. регресси не изменится

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования